# Model training

## Import packages

In [1]:
import tensorflow as tf
import pandas as pd
import os
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import random
import re
import matplotlib.pyplot as plt



In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


2022-05-02 14:02:56.307294: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 14:02:56.354695: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 14:02:56.355205: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


## Read data

In [3]:
# Datensatz einlesen
df = pd.read_csv('data/out.csv')

## Transform dataframe

In [4]:
# Dataframe mit 3 Spalten. Werden so gejoint, dass ein neues Dataframe mit ein Haiku pro Zeile erstellt wird
df = df[['0', '1', '2']].agg(lambda x: '\n'.join(x.values), axis=1)
# Dataframe to list [[]] -> []
array_of_poems = df.values.tolist()

In [5]:
# num_poems an Haikus mit Semicolon zwshceneinander zusammenfuegen
num_poems = 15000

text = ';'.join(array_of_poems[:num_poems])

## Get unique chars in corpus

In [6]:
# Anzahl der unterschielichen characters im gesamt datensatz herausfinden
vocab = sorted(set(text))
vocab_size = len(vocab)
print(f'{vocab_size} unique chars')
print(vocab)

29 unique chars
['\n', ' ', ';', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## Create training batches

In [7]:
print("Total chars:", vocab_size)
# Dictionary erstellen. Jeder character wird mit einer Zahl nummeriert
char_indices = {c: i for i, c in enumerate(vocab)}
indices_char = {i: c for i, c in enumerate(vocab)}

# cut the text in semi-redundant sequences of seq_len characters
seq_len = 100
step = 17
# Input String
sequences = []
#Output character
next_chars = []
for i in range(0, len(text) - seq_len, step):
    sequences.append(text[i : i + seq_len])
    next_chars.append(text[i + seq_len])
print("Number of sequences:", len(sequences))

# Input String onehot encoded
x = np.zeros((len(sequences), seq_len, vocab_size), dtype=int)
# Output char onehot encoded
y = np.zeros((len(sequences), vocab_size), dtype=int)
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


Total chars: 29
Number of sequences: 58171


In [8]:
x.shape[1]

100

In [9]:
x.shape[2]

29

In [10]:
model = keras.Sequential(
    [
        # input_shape=sequenz laenge, vocab_size
        # return sequences true -> input-shape = output-shape 
        # shape-input (NONE, seq_len, vocab_size)
        layers.LSTM(128, input_shape=(x.shape[1], x.shape[2]), return_sequences=True),
        layers.Dropout(0.2),
        # shape-input (NONE, seq_len, vocab_size)
        layers.LSTM(128, return_sequences=True),
        layers.Dropout(0.2),
        # shape-input (NONE, seq_len, vocab_size)
        layers.LSTM(64),
        # shape-input (NONE, vocab_size)
        layers.Dense(vocab_size, activation="softmax"),
        # bsp out [0.3, 0.2, 0.1, 0.4]
    ]
)

optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

2022-05-02 14:03:03.589181: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-02 14:03:03.590433: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 14:03:03.591066: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 14:03:03.591472: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 128)          80896     
                                                                 
 dropout (Dropout)           (None, 100, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 128)          131584    
                                                                 
 dropout_1 (Dropout)         (None, 100, 128)          0         
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 29)                1885      
                                                                 
Total params: 263,773
Trainable params: 263,773
Non-trai

In [12]:
# Standartfunktion Probability array to onehot to integerencoded char 
# [0.3, 0.2, 0.1, 0.4] -> [0, 0, 0, 1] -> return 4 (stelle, an der 1)
def sample(prob, temperature=1.0):
    # helper function to sample an index from a probability array
    prob = np.asarray(prob).astype("float64")
    prob = np.log(prob) / temperature
    exp_prob = np.exp(prob)
    prob = exp_prob / np.sum(exp_prob)
    probas = np.random.multinomial(1, prob)
    return np.argmax(probas)

In [ ]:
epochs = 25
batch_size = 128

input_data = x
output_data = y


for epoch in range(epochs):
    print()
    print()
    print(f"EPOCH:{epoch}")
    model.fit(input_data, output_data, batch_size=batch_size, epochs=1)

    print()

    generate_chars = 200
    temperature = 1.0
    start_index = random.randint(0, len(text) - seq_len - 1)
    generated = ""

    seed =  text[start_index : start_index + seq_len]
    
    #print('...Generating with seed: "' + seed + '"')

    for i in range(generate_chars):
        x_pred = np.zeros((1, len(seed), vocab_size))
        for t, char in enumerate(seed):
            x_pred[0, t, char_indices[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]
        seed = seed[1:] + next_char
        generated += next_char
        
        if next_char == ";":
            generated += "\n----------------------------------------\n"
            
    print(generated) 





EPOCH:0


2022-05-02 14:03:05.543949: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1349567200 exceeds 10% of free system memory.
2022-05-02 14:03:06.957561: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1349567200 exceeds 10% of free system memory.
2022-05-02 14:03:12.516088: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


455/455 [==============================] - 19s 29ms/step - loss: 2.6825

 wyhler ce dilt lewyore weenibe folib cat ke deere phe lafe;
----------------------------------------
doad o gang hhere me mad on the pe ashhaug snertacle the;
----------------------------------------
i g
uke in
sint nhe tt
 le
toa if fu sfaanvp fagarly ghey goled it;
----------------------------------------
torpa
;
----------------------------------------
mon he an


EPOCH:1


2022-05-02 14:03:39.865225: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1349567200 exceeds 10% of free system memory.
2022-05-02 14:03:41.264007: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1349567200 exceeds 10% of free system memory.


455/455 [==============================] - 13s 29ms/step - loss: 2.2763


 thas to thaseltise yore
thing
i uls porreon wanester me;
----------------------------------------
tarefad waot op i heatted
boll
i  drorle;
----------------------------------------
shay he ron berorrerly
cus warkice;
----------------------------------------
gave
 is oke peally will;
----------------------------------------
bet
naniys seeself my nhat to in vur;
----------------------------------------
be
 


EPOCH:2


2022-05-02 14:04:07.000338: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1349567200 exceeds 10% of free system memory.


455/455 [==============================] - 13s 29ms/step - loss: 2.1112

 kleming
goer
all of di
magborice
 you feo asbat and span wad it the west bigh yes as nottion wepa
 iclonsesangred as the frage;
----------------------------------------
beceels is shate anror
theerls onlesch then peoke thes
 beach pirethiul 


EPOCH:3
455/455 [==============================] - 13s 29ms/step - loss: 2.0032

o to fraw in masten your that the
 ank gattle the sever iver at chuzt
 to ile im arrink;
----------------------------------------
have
theme
 a gamp are faxt if whele that chall;
----------------------------------------
every in
shatuare so im chafing can stirther
 right lonest pough


EPOCH:4
455/455 [==============================] - 13s 29ms/step - loss: 1.9220

marie;
----------------------------------------
and noth
rud to;
----------------------------------------
morid;
----------------------------------------

aint to bet ople
 as mat tuck u los belought of cicking
yast the glate no thii

In [ ]:
model.save('myModel.h5')

In [ ]:
model = keras.models.load_model('myModel.h5')

In [ ]:

generate_chars = 300
temperature = 1.0
start_index = random.randint(0, len(text) - seq_len - 1)
generated = ""

seed =  text[start_index : start_index + seq_len]
    
print('...Generating with seed: "' + seed + '"')

    
for i in range(generate_chars):
    x_pred = np.zeros((1, len(seed), vocab_size))
    for t, char in enumerate(seed):
        x_pred[0, t, char_indices[char]] = 1
    preds = model.predict(x_pred, verbose=0)[0]
        
    next_index = sample(preds, temperature)
    next_char = indices_char[next_index]
    seed = seed[1:] + next_char
    generated += next_char
        
    if next_char == ";":
            generated += "\n----------------------------------------\n"
            
print(generated) 
        

